# Cayley-Dickson Construction Applied to Zi Definition

My original implementation of Gaussian integers included two classes, ``Zi`` and ``Qi``, where, for example, ``Zi(2, -7)`` represents a Gaussian integer, and ``Qi(-2/3, 4/5)`` represents a Gaussian rational.

I'd like to extend this code to include integer-valued quaternions and octonions. An elegant way to accomplish that goal would be to use the Cayley-Dickson construction, where complex numbers can be constructed from pairs of real numbers, quaternions can be constructed from pairs of those pairs, and octonions constructed from pairs of those pairs of pairs.

For more specifics, see my write-up about the Cayley-Dickson construction [at this link](https://abstract-algebra.readthedocs.io/en/latest/55_cayley_dickson.html).

In [1]:
# from fractions import Fraction
from random import randint

In [2]:
class Zi:
    """Pairs of integers (Gaussian Integers), Pairs of pairs of integers (Quaternion Integers),
    Pairs of pairs of pairs of integers (Octonion Integers), etc."""
    
    def __init__(self, re=None, im=None):

        # --------------------------------------------------------
        # re is a float or int, and im is a float, int, or None
        
        if isinstance(re, (float, int)):
            self.__re = round(re)
            if im is None:
                self.__im = 0
            elif isinstance(im, (float, int)):
                self.__im = round(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --------------------------------------------------------
        # re is a complex, and im is None, a complex, or a Zi
        
        elif isinstance(re, complex):
            if im is None:  # 
                self.__re = round(re.real)
                self.__im = round(re.imag)
            elif isinstance(im, (complex, Zi)):
                self.__re = Zi(re)
                self.__im = Zi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # --------------------------------------------------------
        # re is a Zi, and im is None, a complex, or a Zi
        
        elif isinstance(re, Zi):
            if im is None:
                self.__re = re.real
                self.__im = re.imag
            elif isinstance(im, (complex, Zi)):
                self.__re = Zi(re)
                self.__im = Zi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")
        
        # --------------------------------------------------------
        # Both re and im are None
        
        elif re is None:
            self.__re = 0
            if im is None:
                self.__im = 0
            else:
                raise Exception(f"If re is None, then im must be None. But im = {im}")
        else:
            raise Exception(f"Unexpected combination of input types: {re} and {im}")
        
    def __repr__(self):
        return f"{self.__class__.__name__}({self.__re}, {self.__im})"
                                
    @property
    def real(self):
        return self.__re
    
    @property
    def imag(self):
        return self.__im

    def __neg__(self):
        return Zi(- self.__re, - self.__im)

    def __add__(self, other):
        return Zi(self.__re + other.real, self.__im + other.imag)

    def __sub__(self, other):
        return Zi(self.__re - other.real, self.__im - other.imag)

    # Cayley-Dickson Construction
    # See [Schafer, 1966]
    # Conjugation (*) define recursively: a* = a and (u, v)* = (u*, -v)
    # Multiplication: (a, b) x (c, d) = (a x c  +  mu x d x b*,  a* x d  +  c x b)
    # where mu is hardcoded below to be -1, for now.
    def __mul__(self, other):
        a, b, c, d = self.__re, self.__im, other.real, other.imag
        real_part = a * c - d * b.conjugate()
        imag_part = a.conjugate() * d + c * b
        return Zi(real_part, imag_part)

    def __rmul__(self, other):
        return self * other

    def conjugate(self):
        return Zi(self.__re, - self.__im)

    def norm(self):
        if isinstance(self.__re, (float, int)) and isinstance(self.__im, (float, int)):
            n = self * self.conjugate()
            return n.real
        elif isinstance(self.__re, Zi) and isinstance(self.__im, Zi):
            p = self.__re.norm() + self.__im.norm()
            return p
        else:
            raise Exception(f"Can't take norm of {self}")
    
    def depth(self):
        """Depth is the number levels contained in the Zi.
        That is, a Zi made up of two integers has depth 0, and a Zi
        made up of two other Zi's, each of depth n, has depth n+1."""
        def aux(x, d):
            if isinstance(x, int):
                return d
            else:
                return aux(x.real, d + 1)
        return aux(self.__re, 0)
            
    def is_complex(self):
        """Return True if this Zi is essentially a complex number
        That is, the re & im parts are numbers, not other Zis."""
        # return isinstance(self.__re, (float, int)) and isinstance(self.__im, (float, int))
        return self.depth() == 0

    def is_quaternion(self):
        """Return True if this Zi is essentially a quaternion
        That is, the re & im parts are essentially complex numbers."""
        # return self.__re.is_complex() and self.__im.is_complex()
        return self.depth() == 1

    def is_octonion(self):
        """Return True if this Zi is essentially an octonion
        That is, the re & im parts are essentially quaternions."""
        # return self.__re.is_quaternion() and self.__im.is_quaternion()
        return self.depth() == 2

    @staticmethod
    def random(re1=-100, re2=100, im1=-100, im2=100):
        """Return a random Zi(re, im) with integer components where,
        re1 <= re <= re2 and im1 <= im <= im2."""
        return Zi(randint(re1, re2), randint(im1, im2))

    # A convenience function for testing purposes:
    def make_complex(self):
        if isinstance(self.__re, (float, int)) and isinstance(self.__im, (float, int)):
            return complex(self.__re, self.__im)
        elif isinstance(self.__re, Zi) and isinstance(self.__im, Zi):
            return Zi(self.make_complex(self.__re), self.make_complex(self.__im))
        else:
            raise Exception(f"Can't make complex out of {self}")

    # --------------------------

    def __radd__(self, other):
        """The reflected (swapped) operand for addition: other + self"""
        return Zi(other) + self

    def __iadd__(self, other):
        """Implements the += operation: self += other"""
        return Zi(self.real + other.real, self.imag + other.imag)

    def __rsub__(self, other):
        """The reflected (swapped) operand for subtraction: other - self"""
        return Zi(other) - self

    def __isub__(self, other):
        """Implements the -= operation: self -= other"""
        return Zi(self.real - other.real, self.imag - other.imag)

    def __imul__(self, other):
        """Implements the *= operation: self *= other"""
        a = self.real
        b = self.imag
        c = round(other.real)
        d = round(other.imag)
        return Zi(a * c - b * d, a * d + b * c)

    def __pow__(self, n: int, modulo=None):
        """Implements the ** operator: self ** n.

        If n == 0, then Zi(1, 0) is returned. If n < 0, then the Gaussian
        rational, Qi, for 1 / self**n is returned. Otherwise, self ** n is returned.
        """
        result = self
        if isinstance(n, int):
            if n == 0:
                result = Zi(1)  # "1"
            elif n > 0:
                for _ in range(n - 1):
                    result = result * self
            else:  # n < 0
                result = 1 / (self ** abs(n))
        else:
            raise TypeError(f"The power, {n}, must be an integer.")
        return result

    def __eq__(self, other) -> bool:
        """Return True if this Zi equals other."""
        return (self.real == other.real) and (self.imag == other.imag)

    def __ne__(self, other) -> bool:
        """Return True if this Zi does NOT equal other."""
        return (self.real != other.real) or (self.imag != other.imag)

    def __hash__(self):
        """Allow this Zi to be hashed."""
        return hash((self.real, self.imag))

    def __abs__(self) -> float:
        """Returns the square root of the norm."""
        return math.sqrt(self.norm)

    def __pos__(self):
        return +self

    def __rpow__(self, base):
        return NotImplemented
    
    def __round__(self):
        if isinstance(self.real, Number) and isinstance(self.imag, Number):
            return Zi(round(self.real), round(self.imag))
        else:
            return self

    @staticmethod
    def eye():
        """Return i = Zi(0, 1)"""
        return Zi(0, 1)

    @staticmethod
    def units():
        """Returns the list of four units, [1, -1, i, -i], as Zis."""
        return [Zi(1), -Zi(1), Zi.eye(), -Zi.eye()]

    @property
    def is_unit(self):
        """Returns True if this Zi is a unit."""
        return self in Zi.units()

    @staticmethod
    def two():
        """Returns 1+i, because a Gaussian integer has an even norm if and only if
        it is a multiple of 1+i."""
        return Zi(1, 1)

    def associates(self):
        """Return a list of this Zi's three associates"""
        us = Zi.units()
        return list(map(lambda u: u * self, us[1:]))  # skip multiplying by 1

    def is_associate(self, other):
        """Return True if the other Zi is an associate of this Zi

        Otherwise, return False.
        """
        q = self // other
        if q:
            if q in Zi.units():
                return True
            else:
                return False
        else:
            return False

In [3]:
Zi()

Zi(0, 0)

In [4]:
Zi(1)

Zi(1, 0)

In [5]:
Zi(1, 2)

Zi(1, 2)

In [6]:
Zi(1.9, 2.1)

Zi(2, 2)

In [7]:
Zi((1.9+2.1j))

Zi(2, 2)

In [8]:
Zi(Zi(1, 2))

Zi(1, 2)

In [9]:
Zi(Zi(), Zi(1))

Zi(Zi(0, 0), Zi(1, 0))

In [10]:
Zi(Zi(3, 4), (1+2j))

Zi(Zi(3, 4), Zi(1, 2))

In [11]:
Zi((3+4j), Zi(1, 2))

Zi(Zi(3, 4), Zi(1, 2))

In [12]:
Zi((3+4j), (1+2j))

Zi(Zi(3, 4), Zi(1, 2))

In [13]:
Zi(Zi(Zi(0), Zi(1)), Zi(Zi(3, 4), Zi(1, 2)))

Zi(Zi(Zi(0, 0), Zi(1, 0)), Zi(Zi(3, 4), Zi(1, 2)))

In [14]:
Zi(Zi(Zi(0), Zi(1)), Zi(Zi(3, 4), (1+2j)))

Zi(Zi(Zi(0, 0), Zi(1, 0)), Zi(Zi(3, 4), Zi(1, 2)))

In [15]:
def random_quaternion(size=10):
    ul = -size; ll = size  # Upper & lower limits of random numbers
    return Zi(Zi.random(ul, ll, ul, ll), Zi.random(ul, ll, ul, ll))

def random_octonion(size=10):
    return Zi(random_quaternion(size), random_quaternion(size))

## Examples

In [16]:
n = 4
zs = [Zi.random(-10, 10, -10, 10) for i in range(n)]
z1 = zs[0]
z2 = zs[1]
z3 = zs[2]
z4 = zs[3]
print(f"{zs = }")
print(f"{z1 = }")
print(f"{z2 = }")
print(f"{z3 = }")
print(f"{z4 = }")

zs = [Zi(-1, 8), Zi(3, -2), Zi(-9, 5), Zi(-3, 6)]
z1 = Zi(-1, 8)
z2 = Zi(3, -2)
z3 = Zi(-9, 5)
z4 = Zi(-3, 6)


In [17]:
print(f"{z1 = }")
print(f"{-z1 = }")
print(f"{z1.real = }")
print(f"{z1.imag = }")
print(f"{z1.conjugate() = }")
print(f"{z1.norm() = }")
print(f"{z1.depth() = }")
print(f"{z1.is_complex() = }")

z1 = Zi(-1, 8)
-z1 = Zi(1, -8)
z1.real = -1
z1.imag = 8
z1.conjugate() = Zi(-1, -8)
z1.norm() = 65
z1.depth() = 0
z1.is_complex() = True


In [18]:
print(f"{z1 = }")
print(f"{z2 = }")
print(f"{z1 + z2 = }")
print(f"{z1 - z2 = }")

z1 = Zi(-1, 8)
z2 = Zi(3, -2)
z1 + z2 = Zi(2, 6)
z1 - z2 = Zi(-4, 10)


For comparisons, create complex numbers corresponding to z1, z2, z3, and z4

In [19]:
c1 = z1.make_complex()
c2 = z2.make_complex()
c3 = z3.make_complex()
c4 = z4.make_complex()

print(f"{z1 = } --> {c1 = }")
print(f"{z2 = } --> {c2 = }")
print(f"{z3 = } --> {c3 = }")
print(f"{z4 = } --> {c4 = }")

z1 = Zi(-1, 8) --> c1 = (-1+8j)
z2 = Zi(3, -2) --> c2 = (3-2j)
z3 = Zi(-9, 5) --> c3 = (-9+5j)
z4 = Zi(-3, 6) --> c4 = (-3+6j)


In [20]:
print(f"{z1 * z2 = }")
print(f"{c1 * c2 = }\n")

print(f"{z1 * 2 = }")
print(f"{2 * z1 = }")

z1 * z2 = Zi(13, 26)
c1 * c2 = (13+26j)

z1 * 2 = Zi(-2, 16)
2 * z1 = Zi(-2, 16)


In [21]:
q1 = Zi(z1, z2)
q2 = Zi(z3, z4)

d1 = Zi(c1, c2)
d2 = Zi(c3, c4)

print(f"{q1.depth() = }")
print(f"{q1.is_complex() = }")
print(f"{q1.is_quaternion() = }")
print(f"{q1 = }")
print(f"{q2 = }\n")

print(f"{d1 = }")
print(f"{d2 = }\n")

print(f"{q1 + q2 = }")
print(f"{q1 * 2 = }")
print(f"{2 * q1 = }")
print(f"{q1 * q2 = }\n")

print(f"{d1 + d2 = }")
print(f"{d1 * d2 = }")

q1.depth() = 1
q1.is_complex() = False
q1.is_quaternion() = True
q1 = Zi(Zi(-1, 8), Zi(3, -2))
q2 = Zi(Zi(-9, 5), Zi(-3, 6))

d1 = Zi(Zi(-1, 8), Zi(3, -2))
d2 = Zi(Zi(-9, 5), Zi(-3, 6))

q1 + q2 = Zi(Zi(-10, 13), Zi(0, 4))
q1 * 2 = Zi(Zi(-2, 16), Zi(6, -4))
2 * q1 = Zi(Zi(-2, 16), Zi(6, -4))
q1 * q2 = Zi(Zi(-10, -89), Zi(34, 51))

d1 + d2 = Zi(Zi(-10, 13), Zi(0, 4))
d1 * d2 = Zi(Zi(-10, -89), Zi(34, 51))


In [22]:
o1 = Zi(q1, q2)
print(f"{o1 = }")
print(f"{o1.depth() = }")
print(f"{o1.is_quaternion() = }")
print(f"{o1.is_octonion() = }")

o1 = Zi(Zi(Zi(-1, 8), Zi(3, -2)), Zi(Zi(-9, 5), Zi(-3, 6)))
o1.depth() = 2
o1.is_quaternion() = False
o1.is_octonion() = True


In [23]:
o1.norm()

229

In [24]:
z1.norm() + z2.norm() + z3.norm() + z4.norm()

229

In [25]:
q1.norm()

78

In [26]:
qx = random_quaternion()
qy = random_quaternion()

In [27]:
qx

Zi(Zi(10, -1), Zi(1, 2))

In [28]:
qy

Zi(Zi(-7, -10), Zi(9, 1))

In [29]:
foo = qx * qx.conjugate()
foo

Zi(Zi(104, -20), Zi(4, -2))

In [30]:
foo.norm()

11236